In [6]:
import pandas as pd
import random
import itertools

idx = [3, 4, 5, 6]
pairs = [(j, 15 - j) for j in idx]

set = []
new_set = []
for m in range(1, 17):
    i = list(range(1, 17))
    i.remove(m)

    for pair in pairs:
        rem_grid = i.copy()

        fst = random.sample(i, pair[0])
        remaining = [cell for cell in i if cell not in fst]
        snd = random.sample(remaining, pair[1])
        remaining = [cell for cell in remaining if cell not in snd]
        grid = [m, fst, snd, remaining]
        grid2 = [m, snd, fst, remaining]
        set.append(grid)
        set.append(grid2)


def fu(grid_set_bias):
    i = 0
    colors = ["red", "blue", "green", "yellow"]
    shape = ["triangle", "square", "circle", "diamond"]
    color_pairs = list(itertools.permutations(colors, 2))
    shape_pairs = list(itertools.permutations(shape, 2))
    for grid in grid_set_bias:
        color_pair = random.choice(color_pairs)
        shape_pair = random.choice(shape_pairs)

        target = (color_pair[0], shape_pair[0])
        color_competitor = (color_pair[0], shape_pair[1])
        shape_competitor = (color_pair[1], shape_pair[0])
        no_competitor = (color_pair[1], shape_pair[1])
        cc_count = len(grid[1])
        sc_count = len(grid[2])
        ratio = cc_count/sc_count
        grid_dict = {
            "Target": target,
            "Target_Position": grid[0],
            "Color_Competitor": color_competitor,
            "Color_Competitor_Position": grid[1],
            "Shape_Competitor": shape_competitor,
            "Shape_Competitor_Position": grid[2],
            "No_Competitor": grid[3],
            "Color/Shape Ratio": ratio 
        }
        new_set.append(grid_dict)
    return new_set

    

a = fu(set)
b = pd.DataFrame(a)
b.to_csv("wup.csv")


In [4]:
b['Color/Shape Ratio'].prod()

np.float64(1.0)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math
import numpy as np
import ast

   

def parse_list(s):
    if isinstance(s, list):
        return s
    try:
        return ast.literal_eval(s)
    except (ValueError, SyntaxError) as e:
        print(f"Error parsing list: {s}, Error: {e}")
        return []

# Validate and process data
df = b.copy()
df['No_Competitor'] = df['No_Competitor'].apply(parse_list)

# Validate data
for idx, row in df.iterrows():
    if not (isinstance(row['Target'], tuple) and len(row['Target']) == 2):
        print(f"Warning: Row {idx} has invalid Target: {row['Target']}")
        continue
    if not (isinstance(row['Color_Competitor'], tuple) and len(row['Color_Competitor']) == 2):
        print(f"Warning: Row {idx} has invalid Color_Competitor: {row['Color_Competitor']}")
        continue
    if not (isinstance(row['Shape_Competitor'], tuple) and len(row['Shape_Competitor']) == 2):
        print(f"Warning: Row {idx} has invalid Shape_Competitor: {row['Shape_Competitor']}")
        continue
    if not isinstance(row['Target_Position'], int) or row['Target_Position'] < 1 or row['Target_Position'] > 16:
        print(f"Warning: Row {idx} has invalid Target_Position: {row['Target_Position']}")
        continue
    if not (isinstance(row['Color_Competitor_Position'], list) and all(isinstance(p, int) and 1 <= p <= 16 for p in row['Color_Competitor_Position'])):
        print(f"Warning: Row {idx} has invalid Color_Competitor_Position: {row['Color_Competitor_Position']}")
        continue
    if not (isinstance(row['Shape_Competitor_Position'], list) and all(isinstance(p, int) and 1 <= p <= 16 for p in row['Shape_Competitor_Position'])):
        print(f"Warning: Row {idx} has invalid Shape_Competitor_Position: {row['Shape_Competitor_Position']}")
        continue
    positions = [row['Target_Position']] + row['Color_Competitor_Position'] + row['Shape_Competitor_Position']
    

# Set up figure
n_grids = len(df)
cols = int(math.ceil(math.sqrt(n_grids)))
rows = int(math.ceil(n_grids / cols))
fig, axes = plt.subplots(rows, cols, figsize=(cols * 4, rows * 4.5), squeeze=False)
axes = axes.flatten()

# Function to draw shapes
def draw_shape(ax, pos, color, shape, grid_size=1):
    row = (pos - 1) // 4
    col = (pos - 1) % 4
    x = col + 0.5
    y = 3 - row + 0.5  # Flip y-axis for top-left as 1
    size = 0.4

    if shape == 'triangle':
        ax.add_patch(patches.Polygon(
            [[x, y + size], [x - size, y - size], [x + size, y - size]],
            facecolor=color, edgecolor='black'
        ))
    elif shape == 'circle':
        ax.add_patch(patches.Circle(
            (x, y), size, facecolor=color, edgecolor='black'
        ))
    elif shape == 'square':
        ax.add_patch(patches.Rectangle(
            (x - size, y - size), size * 2, size * 2, facecolor=color, edgecolor='black'
        ))
    elif shape == 'diamond':
        ax.add_patch(patches.Polygon(
            [[x, y + size], [x + size, y], [x, y - size], [x - size, y]],
            facecolor=color, edgecolor='black'
        ))

# Plot each grid
for idx, (row, ax) in enumerate(zip(df.iterrows(), axes)):
    row = row[1]
    if not (isinstance(row['Target'], tuple) and isinstance(row['Color_Competitor'], tuple) and 
            isinstance(row['Shape_Competitor'], tuple) and isinstance(row['Color_Competitor_Position'], list) and 
            isinstance(row['Shape_Competitor_Position'], list)):
        ax.text(0.5, 2, f'Row {idx} Invalid Data', ha='center', va='center')
        ax.axis('off')
        continue

    ax.set_xlim(0, 4)
    ax.set_ylim(0, 4)
    ax.set_xticks(range(5))
    ax.set_yticks(range(5))
    ax.grid(True)
    ax.set_title(f'Grid {idx + 1}\nRatio: {row["Color/Shape Ratio"]:.3f}')

    # Draw Target
    draw_shape(ax, row['Target_Position'], row['Target'][0], row['Target'][1])

    # Draw Color Competitors
    for pos in row['Color_Competitor_Position']:
        draw_shape(ax, pos, row['Color_Competitor'][0], row['Color_Competitor'][1])

    # Draw Shape Competitors
    for pos in row['Shape_Competitor_Position']:
        draw_shape(ax, pos, row['Shape_Competitor'][0], row['Shape_Competitor'][1])

# Hide unused axes
for ax in axes[len(df):]:
    ax.axis('off')

plt.tight_layout()
plt.savefig('grids.png', dpi=300, bbox_inches='tight')
plt.close()
print("Image saved as 'grids.png'")

Image saved as 'grids.png'


In [12]:
import os
output_dir = 'Tasks'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to draw shapes
def draw_shape(ax, pos, color, shape, grid_size=1):
    row = (pos - 1) // 4
    col = (pos - 1) % 4
    x = col + 0.5
    y = 3 - row + 0.5  # Flip y-axis for top-left as 1
    size = 0.4

    if shape == 'triangle':
        ax.add_patch(patches.Polygon(
            [[x, y + size], [x - size, y - size], [x + size, y - size]],
            facecolor=color, edgecolor='black'
        ))
    elif shape == 'circle':
        ax.add_patch(patches.Circle(
            (x, y), size, facecolor=color, edgecolor='black'
        ))
    elif shape == 'square':
        ax.add_patch(patches.Rectangle(
            (x - size, y - size), size * 2, size * 2, facecolor=color, edgecolor='black'
        ))
    elif shape == 'diamond':
        ax.add_patch(patches.Polygon(
            [[x, y + size], [x + size, y], [x, y - size], [x - size, y]],
            facecolor=color, edgecolor='black'
        ))

for idx, row in df.iterrows():
    if not (isinstance(row['Target'], tuple) and isinstance(row['Color_Competitor'], tuple) and 
            isinstance(row['Shape_Competitor'], tuple) and isinstance(row['Color_Competitor_Position'], list) and 
            isinstance(row['Shape_Competitor_Position'], list)):
        print(f"Skipping row {idx} due to invalid data")
        continue

    # Create figure for single grid
    fig, ax = plt.subplots(figsize=(4, 4.5))

    ax.set_xlim(0, 4)
    ax.set_ylim(0, 4)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xticks(range(5))
    ax.set_yticks(range(5))
    ax.grid(True)
    ax.set_title(f'Grid {idx + 1}\nRatio: {row["Color/Shape Ratio"]:.3f}')
    # Draw Target
    draw_shape(ax, row['Target_Position'], row['Target'][0], row['Target'][1])

    # Draw Color Competitors
    for pos in row['Color_Competitor_Position']:
        draw_shape(ax, pos, row['Color_Competitor'][0], row['Color_Competitor'][1])

    # Draw Shape Competitors
    for pos in row['Shape_Competitor_Position']:
        draw_shape(ax, pos, row['Shape_Competitor'][0], row['Shape_Competitor'][1])

    # Save the image
    output_path = os.path.join(output_dir, f'grid_{idx}.png')
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close(fig)
    print(f"Saved {output_path}")

print(f"All grid images saved in {output_dir}")

Saved Tasks/grid_0.png
Saved Tasks/grid_1.png
Saved Tasks/grid_2.png
Saved Tasks/grid_3.png
Saved Tasks/grid_4.png
Saved Tasks/grid_5.png
Saved Tasks/grid_6.png
Saved Tasks/grid_7.png
Saved Tasks/grid_8.png
Saved Tasks/grid_9.png
Saved Tasks/grid_10.png
Saved Tasks/grid_11.png
Saved Tasks/grid_12.png
Saved Tasks/grid_13.png
Saved Tasks/grid_14.png
Saved Tasks/grid_15.png
Saved Tasks/grid_16.png
Saved Tasks/grid_17.png
Saved Tasks/grid_18.png
Saved Tasks/grid_19.png
Saved Tasks/grid_20.png
Saved Tasks/grid_21.png
Saved Tasks/grid_22.png
Saved Tasks/grid_23.png
Saved Tasks/grid_24.png
Saved Tasks/grid_25.png
Saved Tasks/grid_26.png
Saved Tasks/grid_27.png
Saved Tasks/grid_28.png
Saved Tasks/grid_29.png
Saved Tasks/grid_30.png
Saved Tasks/grid_31.png
Saved Tasks/grid_32.png
Saved Tasks/grid_33.png
Saved Tasks/grid_34.png
Saved Tasks/grid_35.png
Saved Tasks/grid_36.png
Saved Tasks/grid_37.png
Saved Tasks/grid_38.png
Saved Tasks/grid_39.png
Saved Tasks/grid_40.png
Saved Tasks/grid_41.png
Sa